In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib notebook
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
w,h=10,10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Utils

In [3]:
def delete_index(T, index=None):
    if index is None:
        return T
    elif index is -1:
        return T[:index]
    else:
        return torch.cat([T[:index], T[index+1:]])
delete_index(torch.arange(30).reshape(3,10), 1), \
delete_index(torch.arange(30).reshape(3,10)), \
delete_index(torch.arange(30).reshape(3,10), -1)

(tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]]),
 tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]]),
 tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]))

In [4]:
def delete_indexes(T, indexes=None):
    if indexes is None:
        return T
    else:
        if indexes[0] > indexes[1]: # ex indexes = [0, -1]
            return T
        elif indexes[1] == -1: # ex indexes = [-3, -1]
            return T[:indexes[0]]
        return torch.cat([T[:indexes[0]], T[indexes[1]+1:]])
arr = torch.arange(15).reshape(5,3)
arr, delete_indexes(arr, [0,2])

(tensor([[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11],
         [12, 13, 14]]),
 tensor([[ 9, 10, 11],
         [12, 13, 14]]))

In [5]:
def fifo(current_arr, arr):
    current_arr = torch.unsqueeze(current_arr, 0)
    arr = delete_index(arr, index=0)
    return torch.cat((arr, current_arr))
fifo(torch.arange(10), torch.zeros(30).reshape(3,10))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [6]:
def push(arr, current_arr):
    if arr is None:
        return torch.unsqueeze(current_arr, 0)
    current_arr = torch.unsqueeze(current_arr, 0)
    return torch.cat((arr, current_arr))
push(torch.zeros(10).reshape(1,10), torch.arange(10)), push(None, torch.arange(10))

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]),
 tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]))

In [7]:
def jump_bound(position):
    if position[0] >= w:
        position[0] = position[0] - w
    elif position[0] < 0:
        position[0] = w + position[0]
    if position[1] >= h:
        position[1] = position[1] - w
    elif position[1] < 0:
        position[1] = h + position[1]
    return position
jump_bound([10, -1])

[0, 9]

### Env

In [8]:
class MomentumObject:
    def __init__(self, w, h, device):
        self.w, self.h = w,h
        self._position = torch.zeros((2), dtype=torch.int).to(device)
        self._speed =  torch.zeros((2), dtype=torch.int).to(device)
    
    @property
    def position(self):
        return torch.clone(self._position)
    @property
    def speed(self):
        return torch.clone(self._speed)
    @speed.setter
    def speed(self, speed):
        self._speed = speed
    def step(self):
        self._position += self._speed
        jump_bound(self._position)
    def render(self, predict_position=None):
        state = torch.zeros((w,h))
        state[self._position[0], self._position[1]] = 1
        if predict_position is not None:
            predict_position = predict_position.type(torch.int)
            state[predict_position[0], predict_position[1]] = 0.2
        return state
momentum_object = MomentumObject(w, h, device)
momentum_object.step()
momentum_object.render()

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

### Add to Buffer

In [62]:
class MemoryReplay:
    def __init__(self, num_step_per_action, num_action_buffer, num_state, num_action, env):
        self.num_step_per_action = num_step_per_action
        self.num_action_buffer = num_action_buffer
        self.env = env
        self._states_actions = torch.zeros((num_step_per_action*num_action_buffer, num_state+num_action), dtype=torch.double).to(device)
        self.current_step = -1
        self._num_buffer = self.num_step_per_action * self.num_action_buffer
    def step(self):
        self.current_step += 1
        if self.current_step >= num_step_per_action:
            self.current_step = 0
        self.env.step()
        if self.current_step % self.num_step_per_action == 0:
            self.env.speed = torch.randint(-2, 2, size=(2,), device=device)
        state_action = torch.cat((self.env.position, self.env.speed))
        self._states_actions = fifo(state_action, self._states_actions)
    @property
    def states_actions(self):
        return torch.clone(self._states_actions)
    def warmup(self):
        for _ in range(self._num_buffer):
            self.step()
num_step_per_action = 4
num_action_buffer = 3
memory = MemoryReplay(num_step_per_action=num_step_per_action, 
                      num_action_buffer=num_action_buffer, 
                      num_state=2, 
                      num_action=2, 
                      env=momentum_object)
memory.warmup()

In [63]:
memory.step()
memory.step()
memory.states_actions, memory.current_step

(tensor([[ 8.,  5., -1., -1.],
         [ 7.,  4., -1., -1.],
         [ 6.,  3., -1., -2.],
         [ 5.,  1., -1., -2.],
         [ 4.,  9., -1., -2.],
         [ 3.,  7., -1., -2.],
         [ 2.,  5., -2.,  1.],
         [ 0.,  6., -2.,  1.],
         [ 8.,  7., -2.,  1.],
         [ 6.,  8., -2.,  1.],
         [ 4.,  9.,  0., -1.],
         [ 4.,  8.,  0., -1.]], device='cuda:0', dtype=torch.float64),
 1)

##### Find index shift (after add repeated indexes), from "START" or "END"

In [64]:
def get_index_shift(shift, from_start):
    if from_start:
        return shift*num_step_per_action - torch.sum(torch.arange(shift))
    else:
        return torch.sum(torch.arange(shift+1))
get_index_shift(1, True), get_index_shift(2, True), get_index_shift(1, False), get_index_shift(3, False)

(tensor(4), tensor(7), tensor(1), tensor(6))

##### Map direct indexes to repeated indexes

In [122]:
def map_increase_indexes(action_group_id, increment_index):
    x = torch.arange(num_step_per_action)
    repeats = torch.flip(x, dims=(0,)) + 1
    current_indexes = torch.repeat_interleave(x, repeats=repeats, dim=0)
    if increment_index:
        current_indexes += (action_group_id*num_step_per_action)
    return current_indexes
map_increase_indexes(0, True), map_increase_indexes(1, True)

(tensor([0, 0, 0, 0, 1, 1, 1, 2, 2, 3]),
 tensor([4, 4, 4, 4, 5, 5, 5, 6, 6, 7]))

In [121]:
[torch.arange(i) for i in range(num_step_per_action, 0, -1)]

[tensor([0, 1, 2, 3]), tensor([0, 1, 2]), tensor([0, 1]), tensor([0])]

In [123]:
def get_target_index(increment_index=True):
    # calculate shift after repeat target
    shift_start = (memory.current_step+1)%num_step_per_action
    shift_end = num_step_per_action - shift_start
    start_index = get_index_shift(shift_start, True)
    end_index = get_index_shift(shift_end, False)
    
    # repeat target indexes
    current_indexes = [map_indexes(i, increment_index) for _, i in enumerate(range(num_action_buffer + 1))]
    current_indexes = torch.cat(current_indexes).to(device)
#     current_indexes_raw = current_indexes
    current_indexes = delete_indexes(current_indexes, [0, start_index-1])
    current_indexes = delete_indexes(current_indexes, [-end_index, -1])
    return current_indexes-shift_start
current_indexes = get_target_index()
current_indexes

tensor([ 0,  1,  1,  1,  1,  2,  2,  2,  3,  3,  4,  5,  5,  5,  5,  6,  6,  6,
         7,  7,  8,  9,  9,  9,  9, 10, 10, 10, 11, 11], device='cuda:0')

In [124]:
def get_repeat_target_state(state):
    current_indexes = get_target_index(increment_index=True)
    return torch.index_select(state, dim=0, index=current_indexes)
target_states = get_repeat_target_state(memory.states_actions[:,:2])
target_states

tensor([[3., 6.],
        [2., 7.],
        [2., 7.],
        [2., 7.],
        [2., 7.],
        [1., 7.],
        [1., 7.],
        [1., 7.],
        [0., 7.],
        [0., 7.],
        [9., 7.],
        [8., 7.],
        [8., 7.],
        [8., 7.],
        [8., 7.],
        [9., 8.],
        [9., 8.],
        [9., 8.],
        [0., 9.],
        [0., 9.],
        [1., 0.],
        [2., 1.],
        [2., 1.],
        [2., 1.],
        [2., 1.],
        [3., 1.],
        [3., 1.],
        [3., 1.],
        [4., 1.],
        [4., 1.]], device='cuda:0', dtype=torch.float64)

In [125]:
def get_repeat_target_state_with_delta_time(state):
    state = get_repeat_target_state(state)
    delta_time = get_target_index(increment_index=False)
    delta_time = torch.unsqueeze(delta_time, 1)
    return torch.cat((state, delta_time), dim=1)
get_repeat_target_state_with_delta_time(memory.states_actions), memory.states_actions.shape

(tensor([[ 3.,  6., -1.,  1.,  0.],
         [ 2.,  7., -1.,  0., -3.],
         [ 2.,  7., -1.,  0., -3.],
         [ 2.,  7., -1.,  0., -3.],
         [ 2.,  7., -1.,  0., -3.],
         [ 1.,  7., -1.,  0., -2.],
         [ 1.,  7., -1.,  0., -2.],
         [ 1.,  7., -1.,  0., -2.],
         [ 0.,  7., -1.,  0., -1.],
         [ 0.,  7., -1.,  0., -1.],
         [ 9.,  7., -1.,  0.,  0.],
         [ 8.,  7.,  1.,  1., -3.],
         [ 8.,  7.,  1.,  1., -3.],
         [ 8.,  7.,  1.,  1., -3.],
         [ 8.,  7.,  1.,  1., -3.],
         [ 9.,  8.,  1.,  1., -2.],
         [ 9.,  8.,  1.,  1., -2.],
         [ 9.,  8.,  1.,  1., -2.],
         [ 0.,  9.,  1.,  1., -1.],
         [ 0.,  9.,  1.,  1., -1.],
         [ 1.,  0.,  1.,  1.,  0.],
         [ 2.,  1.,  1.,  0., -3.],
         [ 2.,  1.,  1.,  0., -3.],
         [ 2.,  1.,  1.,  0., -3.],
         [ 2.,  1.,  1.,  0., -3.],
         [ 3.,  1.,  1.,  0., -2.],
         [ 3.,  1.,  1.,  0., -2.],
         [ 3.,  1.,  1.,  0.

### SD + Normalize function

In [69]:
def standardize_state(state):
    ''' 
    state: [0, 9], [0, 9], action1, action2
    return: [-4.5, 4.5], [-4.5, 4.5], action1, action2
    '''
    state = torch.clone(state)
    state[:, 0] = state[:, 0] - ((w-1)/2)
    state[:, 1] = state[:, 1] - ((h-1)/2)
    return state
def normalize_state(state):
    ''' 
    state: [0, 9], [0, 9]
    return [0, 1], [0, 1] 
    '''
    state = torch.clone(state)
    state[:, 0] = (state[:, 0] + 0.5) / w
    state[:, 1] = (state[:, 1] + 0.5) / h
    return state
def denormalize_state(state):
    ''' 
    state: [0, 1], [0, 1] 
    return [0, 9], [0, 9]
    '''
    state = torch.clone(state)
    state[:, 0] = state[:, 0]*w -0.5
    state[:, 1] = state[:, 1]*h -0.5
    return state
a = denormalize_state(torch.tensor([[0.,1.]]))
a, torch.round(a)
# normalize_state(torch.tensor([[0.,9.]]))

(tensor([[-0.5000,  9.5000]]), tensor([[-0., 10.]]))

### Train

In [ ]:
def train(i):
    memory.step()
    encoder.train()
    optimizer.zero_grad()
    current_state = standardize_state(memory.states_actions)
    predict_state = encoder(current_state)
    
    state_prediction_feature = get_repeat_target_state_with_delta_time(predict_state)
    state_prediction = decoder(state_prediction_feature)
    
    future_label = normalize_state(memory.states_actions[:,:2])
    future_label = get_repeat_target_state(future_label)
    loss = mse_loss(state_prediction, future_label)
    loss.mean().backward(retain_graph=True)
    
    state_prediction = torch.round(denormalize_state(state_prediction))
    miss_distance = (future_label - state_prediction).float().sum().cpu().detach().numpy()
    print("loss[", i,"]:", loss.mean().cpu().detach().numpy(), 'miss_distance', miss_distance)
    
    loss.mean().backward()
    optimizer.step()
    
torch.manual_seed(1)
encoder = nn.Sequential(
                nn.Linear(4,16),
                nn.ReLU(),
                nn.Linear(16,32),
                nn.ReLU(),
                nn.Linear(32,8),
                nn.ReLU(),
                nn.Linear(8,8),
                nn.Sigmoid()).to(device).to(torch.float64)
decoder = nn.Sequential(
                nn.Linear(9,16),
                nn.ReLU(),
                nn.Linear(16,32),
                nn.ReLU(),
                nn.Linear(32,8),
                nn.ReLU(),
                nn.Linear(8,2),
                nn.Sigmoid()).to(device).to(torch.float64)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.001)
mse_loss = nn.MSELoss(reduction='none')

lowest_loss_index = None
for i in range(5000):
    lowest_loss_index = train(i)

loss[ 0 ]: 0.1027811355762795 miss_distance -211.0
loss[ 1 ]: 0.10489019228652034 miss_distance -211.79999
loss[ 2 ]: 0.10206551540851362 miss_distance -212.7
loss[ 3 ]: 0.09916612556331074 miss_distance -213.5
loss[ 4 ]: 0.09887737083488976 miss_distance -214.99998
loss[ 5 ]: 0.10104762057390775 miss_distance -217.4
loss[ 6 ]: 0.09647392600108298 miss_distance -219.2
loss[ 7 ]: 0.08886055939098303 miss_distance -220.4
loss[ 8 ]: 0.08618215909283311 miss_distance -220.0
loss[ 9 ]: 0.07883235264305993 miss_distance -218.4
loss[ 10 ]: 0.08072677302692002 miss_distance -218.4
loss[ 11 ]: 0.08213030463270103 miss_distance -217.20001
loss[ 12 ]: 0.08206500727096597 miss_distance -217.0
loss[ 13 ]: 0.07907577014243942 miss_distance -217.8
loss[ 14 ]: 0.07897956977755254 miss_distance -218.1
loss[ 15 ]: 0.07811987288461048 miss_distance -218.1
loss[ 16 ]: 0.07743458051315354 miss_distance -218.0
loss[ 17 ]: 0.07700261899373773 miss_distance -221.2
loss[ 18 ]: 0.07147371723990827 miss_distance

loss[ 155 ]: 0.12075925806612044 miss_distance -208.6
loss[ 156 ]: 0.11790410690427378 miss_distance -194.0
loss[ 157 ]: 0.10873492205566153 miss_distance -190.40001
loss[ 158 ]: 0.10553041792647215 miss_distance -189.5
loss[ 159 ]: 0.1056206655459575 miss_distance -191.09999
loss[ 160 ]: 0.10600859423525757 miss_distance -192.0
loss[ 161 ]: 0.08888378461484102 miss_distance -192.0
loss[ 162 ]: 0.08082224432084122 miss_distance -191.0
loss[ 163 ]: 0.08008226012928385 miss_distance -191.0
loss[ 164 ]: 0.08159295657170115 miss_distance -194.0
loss[ 165 ]: 0.07890690813779892 miss_distance -197.0
loss[ 166 ]: 0.07805803331666784 miss_distance -197.0
loss[ 167 ]: 0.08053281550557019 miss_distance -197.0
loss[ 168 ]: 0.08310043243187232 miss_distance -198.0
loss[ 169 ]: 0.091354671993736 miss_distance -197.0
loss[ 170 ]: 0.10307621601873275 miss_distance -197.70001
loss[ 171 ]: 0.10699779967583829 miss_distance -201.29999
loss[ 172 ]: 0.10641683768608462 miss_distance -198.0
loss[ 173 ]: 0.

loss[ 320 ]: 0.06875825708243817 miss_distance -210.00002
loss[ 321 ]: 0.07699058739365232 miss_distance -204.40001
loss[ 322 ]: 0.0855457201022946 miss_distance -213.1
loss[ 323 ]: 0.08513758319041292 miss_distance -220.5
loss[ 324 ]: 0.07977404425038809 miss_distance -212.0
loss[ 325 ]: 0.08103871148744526 miss_distance -236.2
loss[ 326 ]: 0.08085418278634518 miss_distance -265.6
loss[ 327 ]: 0.07142407530274193 miss_distance -279.2
loss[ 328 ]: 0.06696981117001541 miss_distance -270.0
loss[ 329 ]: 0.062920731042409 miss_distance -284.19998
loss[ 330 ]: 0.059487515120820615 miss_distance -272.9
loss[ 331 ]: 0.05601209835924675 miss_distance -272.90002
loss[ 332 ]: 0.047978905457931584 miss_distance -255.0
loss[ 333 ]: 0.03769995868212367 miss_distance -223.8
loss[ 334 ]: 0.025661831849475507 miss_distance -193.40001
loss[ 335 ]: 0.021345502553801965 miss_distance -200.79999
loss[ 336 ]: 0.01736741800329152 miss_distance -204.0
loss[ 337 ]: 0.01916296485685886 miss_distance -193.8
los

loss[ 488 ]: 0.041351801816186896 miss_distance -223.0
loss[ 489 ]: 0.033584351786879754 miss_distance -229.4
loss[ 490 ]: 0.02490406768605736 miss_distance -241.29999
loss[ 491 ]: 0.02694618634144441 miss_distance -252.3
loss[ 492 ]: 0.02812134571483793 miss_distance -255.0
loss[ 493 ]: 0.027895782132281017 miss_distance -250.6
loss[ 494 ]: 0.02328338736985202 miss_distance -247.2
loss[ 495 ]: 0.020791255727894167 miss_distance -232.20001
loss[ 496 ]: 0.02067365695679911 miss_distance -224.0
loss[ 497 ]: 0.024556707586694433 miss_distance -224.4
loss[ 498 ]: 0.027986442017475136 miss_distance -223.79999
loss[ 499 ]: 0.030574292537994863 miss_distance -219.8
loss[ 500 ]: 0.030830692509977286 miss_distance -227.0
loss[ 501 ]: 0.033666543048801706 miss_distance -235.6
loss[ 502 ]: 0.03232307292983577 miss_distance -240.1
loss[ 503 ]: 0.02693245851660273 miss_distance -239.29999
loss[ 504 ]: 0.023435522060919173 miss_distance -244.0
loss[ 505 ]: 0.022756907601375075 miss_distance -235.200

loss[ 649 ]: 0.0603589837254113 miss_distance -260.8
loss[ 650 ]: 0.05955371333765331 miss_distance -263.8
loss[ 651 ]: 0.05913945400907865 miss_distance -265.4
loss[ 652 ]: 0.059208985220281285 miss_distance -257.0
loss[ 653 ]: 0.059034746818373396 miss_distance -262.6
loss[ 654 ]: 0.05635910815780898 miss_distance -258.4
loss[ 655 ]: 0.05185612722082823 miss_distance -257.0
loss[ 656 ]: 0.04749915541433321 miss_distance -251.0
loss[ 657 ]: 0.037653736484661984 miss_distance -252.80002
loss[ 658 ]: 0.024491005401924133 miss_distance -235.69998
loss[ 659 ]: 0.018729514680779442 miss_distance -225.5
loss[ 660 ]: 0.016961381762520645 miss_distance -227.0
loss[ 661 ]: 0.015002894265918575 miss_distance -201.4
loss[ 662 ]: 0.014742795542312693 miss_distance -188.59999
loss[ 663 ]: 0.014806120163136443 miss_distance -184.0
loss[ 664 ]: 0.014159180708960906 miss_distance -185.0
loss[ 665 ]: 0.016462314120333627 miss_distance -158.2
loss[ 666 ]: 0.014350657728752209 miss_distance -158.9
loss[

...

loss[ 3879 ]: 0.0002444395384677835 miss_distance -298.5
loss[ 3880 ]: 0.0003012482462017691 miss_distance -302.0
loss[ 3881 ]: 0.00025570009768454197 miss_distance -317.4
loss[ 3882 ]: 0.00033518891012751295 miss_distance -300.9
loss[ 3883 ]: 0.00026032668231634596 miss_distance -296.69998
loss[ 3884 ]: 0.0004428308322410445 miss_distance -295.0
loss[ 3885 ]: 0.0003609577237202753 miss_distance -287.8
loss[ 3886 ]: 0.00025618758194934175 miss_distance -279.69998
loss[ 3887 ]: 0.00028648298552142875 miss_distance -271.5
loss[ 3888 ]: 0.00022588294012204215 miss_distance -267.0
loss[ 3889 ]: 0.0004368054341645179 miss_distance -245.4
loss[ 3890 ]: 0.00038472799046943376 miss_distance -240.0
loss[ 3891 ]: 0.00038000047053758255 miss_distance -243.6
loss[ 3892 ]: 0.00026973182649571556 miss_distance -241.0
loss[ 3893 ]: 0.00035251436895277714 miss_distance -241.0
loss[ 3894 ]: 0.0004910087150791615 miss_distance -246.4
loss[ 3895 ]: 0.0004546532197347902 miss_distance -253.6
loss[ 3896 ]:

loss[ 4038 ]: 8.215138918520837e-05 miss_distance -219.29999
loss[ 4039 ]: 8.265431805441067e-05 miss_distance -244.5
loss[ 4040 ]: 0.00011245826766109176 miss_distance -240.0
loss[ 4041 ]: 9.808737316594858e-05 miss_distance -225.6
loss[ 4042 ]: 0.00012551030290886263 miss_distance -212.1
loss[ 4043 ]: 0.0001323587990900686 miss_distance -219.3
loss[ 4044 ]: 0.000120139344400171 miss_distance -222.0
loss[ 4045 ]: 9.84110477551565e-05 miss_distance -193.20001
loss[ 4046 ]: 0.00019550503611203527 miss_distance -177.3
loss[ 4047 ]: 0.00020761472042343503 miss_distance -181.5
loss[ 4048 ]: 0.00021238991995636825 miss_distance -186.0
loss[ 4049 ]: 0.00016894504062649426 miss_distance -207.6
loss[ 4050 ]: 0.00018050762125587297 miss_distance -194.1
loss[ 4051 ]: 0.00019564483512396813 miss_distance -183.3
loss[ 4052 ]: 0.0002018047902375843 miss_distance -186.0
loss[ 4053 ]: 0.00015463801084174508 miss_distance -229.2
loss[ 4054 ]: 0.0001597929613531391 miss_distance -261.6
loss[ 4055 ]: 0.

loss[ 4188 ]: 0.00025656157817682 miss_distance -168.0
loss[ 4189 ]: 0.0002416694265465026 miss_distance -189.6
loss[ 4190 ]: 0.00031101183354746134 miss_distance -205.79999
loss[ 4191 ]: 0.0002849240423126532 miss_distance -216.6
loss[ 4192 ]: 0.0002762317061038999 miss_distance -220.99998
loss[ 4193 ]: 0.00023406230396446105 miss_distance -243.59999
loss[ 4194 ]: 0.00023045257301453079 miss_distance -254.40001
loss[ 4195 ]: 0.00019390955461915649 miss_distance -276.0
loss[ 4196 ]: 0.0001448928936011192 miss_distance -276.0
loss[ 4197 ]: 0.00015106800781880482 miss_distance -268.8
loss[ 4198 ]: 0.0001589781334352222 miss_distance -274.2
loss[ 4199 ]: 0.0001485837634515452 miss_distance -285.0
loss[ 4200 ]: 0.000146141614964594 miss_distance -285.0
loss[ 4201 ]: 0.000135480602708139 miss_distance -299.4
loss[ 4202 ]: 0.00015358426390600332 miss_distance -283.19998
loss[ 4203 ]: 0.00019955170805306248 miss_distance -272.40002
loss[ 4204 ]: 0.00018682735728020624 miss_distance -267.0
los

loss[ 4342 ]: 0.0005850837447848018 miss_distance -216.0
loss[ 4343 ]: 0.0006374979879189796 miss_distance -207.19998
loss[ 4344 ]: 0.0005139460747460006 miss_distance -195.0
loss[ 4345 ]: 0.0003830953525006126 miss_distance -195.0
loss[ 4346 ]: 0.0004441643170239881 miss_distance -206.1
loss[ 4347 ]: 0.000445562078111632 miss_distance -216.9
loss[ 4348 ]: 0.0003870510953211416 miss_distance -222.0
loss[ 4349 ]: 0.00042186584072735015 miss_distance -265.2
loss[ 4350 ]: 0.0002906028101140004 miss_distance -251.7
loss[ 4351 ]: 0.0002570729051306878 miss_distance -248.1
loss[ 4352 ]: 0.00028175718117033296 miss_distance -249.0
loss[ 4353 ]: 0.00026623058442654937 miss_distance -263.4
loss[ 4354 ]: 0.0002543774753915564 miss_distance -271.5
loss[ 4355 ]: 0.0002594699644517613 miss_distance -275.09998
loss[ 4356 ]: 0.00015108576330830005 miss_distance -276.0
loss[ 4357 ]: 0.00015372593435491696 miss_distance -312.0
loss[ 4358 ]: 0.00021859330427484645 miss_distance -330.9
loss[ 4359 ]: 0.00

loss[ 4497 ]: 0.00020658865386048836 miss_distance -295.8
loss[ 4498 ]: 0.00021093447914793568 miss_distance -285.0
loss[ 4499 ]: 0.00022402427594940633 miss_distance -274.2
loss[ 4500 ]: 0.00023074546660117404 miss_distance -267.0
loss[ 4501 ]: 0.00022476836334307646 miss_distance -303.0
loss[ 4502 ]: 0.00022509700073950172 miss_distance -273.3
loss[ 4503 ]: 0.00021851608744816498 miss_distance -251.70001
loss[ 4504 ]: 0.00019305861880982542 miss_distance -249.0
loss[ 4505 ]: 0.00016942496427622907 miss_distance -234.6
loss[ 4506 ]: 0.00019480151294223863 miss_distance -223.8
loss[ 4507 ]: 0.0002159375621341707 miss_distance -216.6
loss[ 4508 ]: 0.000181457253416794 miss_distance -213.0
loss[ 4509 ]: 0.00018503305906985556 miss_distance -198.6
loss[ 4510 ]: 0.00027853056438228513 miss_distance -193.20001
loss[ 4511 ]: 0.00026467066095037207 miss_distance -193.2
loss[ 4512 ]: 0.00024845888549463835 miss_distance -195.0
loss[ 4513 ]: 0.00024875487870916485 miss_distance -137.4
loss[ 451

### Visualize

In [ ]:
fig = plt.figure()
im = plt.imshow(momentum_object.render(), animated=True)
def updatefig(*args):
    position = momentum_object.get_state_action().reshape(1,4)
    predict_position = encoder(standardize_state(position)).detach()
    _predict_position = torch.round(denormalize_state(predict_position).detach())[0]
    momentum_object.step()
    im.set_array(momentum_object.render(_predict_position))
ani = FuncAnimation(fig, updatefig, interval=500, frames=100, blit=True)
plt.show()
updatefig()

In [109]:
memory.step()
position = memory.states_actions[0].reshape(1,4)
num_next_step = num_step_per_action - memory.current_step
predict_position = encoder(standardize_state(position)).detach()
predict_position = predict_position.repeat((num_next_step, 1))
predict_position_delta_time = torch.cat((predict_position, 
                               torch.arange(num_next_step, device=device).reshape(-1,1)), 
                               axis=1)
torch.round(denormalize_state(decoder(predict_position_delta_time)).detach())

tensor([[8., 3.]], device='cuda:0', dtype=torch.float64)